In [1]:
# This websocket consumer is also a kafak producer


import asyncio
import logging
import websockets
import json
import environ
import pandas as pd
from kafka import KafkaAdminClient
from kafka.producer import KafkaProducer

bitcoin_data = pd.DataFrame.from_dict(
    {
        "time_exhange": [],
        "time_coinapi": [],
        "uuid": [],
        "price": [],
        "size": [],
        "taker_side": [],
        "symbol_id": [],
        "sequence": [],
        "type": [],
    }
)


env = environ.Env(
    # set casting, default value
    DEBUG=(bool, False)
)

environ.Env.read_env()





/home/arun/.virtualenvs/real_time_crypto_prediction/lib/python3.8/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_26080/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


In [2]:
producer = KafkaProducer(bootstrap_servers="localhost:9092,localhost:9093,locahost:9094")

In [3]:

with open("/home/arun/WORK/real-time-crypto-prediction/message.json", "r") as json_file:
    message_dict = json.load(json_file)

logging.basicConfig(level=logging.INFO)


def log_message(message: str) -> None:
    logging.info(f"Message:- {message}")


async def consumer_handler(websocket) -> None:
    async for message in websocket:
        # log_message(type(message))
        response_dict = json.loads(message)

        if response_dict["symbol_id"] == "COINBASE_SPOT_BTC_USD":
            log_message(f"Bitcoin Symbol detected, Price:- {response_dict}")
            # bitcoin_data.append(response_dict, ignore_index=True)
            producer.send(
                topic='bitcoin',
                key=response_dict["uuid"].encode("utf-8"),
                value=message.encode("utf-8"),
            )

            bitcoin_temp_df = pd.DataFrame.from_dict(response_dict)
            pd.merge([bitcoin_data,bitcoin_temp_df], axis = 1)

            # log_message(response_dict.keys())


async def consume(url: str) -> None:

    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(message_dict))

        await consumer_handler(websocket)


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(consume(url="wss://ws.coinapi.io/v1/"))
    loop.run_forever()


RuntimeError: This event loop is already running

INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9093 <connecting> [IPv4 ('127.0.0.1', 9093)]>: connecting to localhost:9093 [('127.0.0.1', 9093) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9093 <connecting> [IPv4 ('127.0.0.1', 9093)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-2 host=localhost:9093 <connected> [IPv4 ('127.0.0.1', 9093)]>: Closing connection. 
